In [6]:
import os
from dotenv import load_dotenv
load_dotenv()

pinecone_api = os.getenv('PINECONE_API_KEY')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')

In [2]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [3]:
from pinecone import Pinecone, ServerlessSpec
index_name = 'hybridsearch-pinecone-langchain'

#init Pinecone client
pc = Pinecone(api_key=pinecone_api)

#create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric='dotproduct',
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )

In [4]:
index = pc.Index(index_name)
index

In [7]:
#vector embedding and sparse matrix
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [8]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()
bm25_encoder

In [9]:
sentences = [
    "In 2024, I visit Munich",
    "In 2023, I visit Germany"
]

#apply tf-idf
bm25_encoder.fit(sentences)

#store values to a json file
bm25_encoder.dump('bm25_values.json')

100%|██████████| 2/2 [00:00<00:00, 40.58it/s]


In [10]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder=bm25_encoder, index=index)